# RANZCR CLiP
- [Basecode](https://www.kaggle.com/sohomdey/ranzcr-clip-efficientnet-auc-0-95-tpu)


### Accelerator
- TPU v3-8 [[doc](https://www.kaggle.com/docs/tpu)]  
    - 참고 코드
        - https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training/data?select=tf_keras_efficientnet.py
        - https://www.kaggle.com/bbalrangco/five-flowers-on-tpu-with-new-data-augmentation/edit

### Module version
- Seaborn version 0.10.0
- Pandas version 1.1.5
- Numpy version 1.17.5
- cv2 version 4.41
- Tenserflow version 2.4.1
- Sklearn version 0.24.1
- Tenserflow.keras version 2.4.0

### Database
- MongoDB server version 4.4.3
- Python 3.9.1
- Pycharm 2020.3.3
- mongoDB Compass 1.25.0

## Workflow

* Image preprocessing
    * (present) Current architecture requires 3 channel inputs, need to fix it.
    * (later)   Image preprocessing to improve the clarity of images
    * (later)   More augmentation
    * (later)   Different image sizes
* Class balancing
    * Weighted Loss Functions $\rightarrow$ tf.keras.callbacks.ReduceLROnPlateau
    * Oversampling $\rightarrow$ tf.keras.callbacks.EarlyStopping
* Architecture tuning
* Ensembling

## Install

In [ ]:
# >> /dev/null : 표준 오류 출력만 무시
! pip install -q efficientnet >> /dev/null

## Import modules

- 평가 메트릭 AUC : tf.keras.metrics.AUC
- 교차 검증 : KFold (미사용)
- [mixed_precision](https://www.tensorflow.org/guide/mixed_precision?hl=ko) : 훈련 중에 모델에서 16-bit 및 32-bit 부동 소수점 유형을 모두 사용하여 더 빠르게 실행하고 메모리를 적게 사용 (TPU 환경 'bfloat16' 사용)
- backend : [tensorflow.keras.backend 사용](https://i-am-eden.tistory.com/2)

In [ ]:
import os
import re
import math
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.mixed_precision import experimental as mixed_precision

from sklearn.model_selection import KFold
# from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix

import efficientnet.tfkeras as efn

## Set path

In [ ]:
DATA_PATH = '/kaggle/input/ranzcr-clip-catheter-line-classification'

MODEL_PATH = '/kaggle/working/models'

In [ ]:
#tf record 파일 갯수 확인

NUM_TF_RECS = len(os.listdir(f'{DATA_PATH}/train_tfrecords'))

print(NUM_TF_RECS)

16


## Set parameters

In [ ]:
DEVICE = 'TPU' # ['CPU' GPU' 'TPU']

ENABLE_MIXED_PRECISION = True # [True False]

In [ ]:
SEED = 42

FOLDS = 5 

EFF_NET = 'B0' # ['B0',B1','B2',B3','B4',B5','B6',B7']
# effnets = {'B0': efn.EfficientNetB0,'B1': efn.EfficientNetB1,'B2': efn.EfficientNetB2,'B3': efn.EfficientNetB3,'B4': efn.EfficientNetB4,'B5': efn.EfficientNetB5,'B6': efn.EfficientNetB6,'B7': efn.EfficientNetB7}

if EFF_NET=='B0':
    IMG_SIZE = 224
elif EFF_NET=='B1':
    IMG_SIZE = 240
elif EFF_NET=='B2':
    IMG_SIZE = 260
elif EFF_NET=='B3':
    IMG_SIZE = 300
elif EFF_NET=='B4':
    IMG_SIZE = 380
elif EFF_NET=='B5':
    IMG_SIZE = 456
elif EFF_NET=='B6':
    IMG_SIZE = 528
elif EFF_NET=='B7':
    IMG_SIZE = 600


BATCH_SIZE = 32 # [8, 16, 32, 64, 128, 256, 512]

EPOCHS = 25

VERBOSE = 2 # [0: silent, 1: progress bar, 2: single line]

## Setup devices and settings

In [ ]:
# For kaggle tpus (Need Internet)
from kaggle_datasets import KaggleDatasets
if DEVICE == 'TPU':
    DATA_PATH = KaggleDatasets().get_gcs_path(DATA_PATH.split('/')[-1])

In [ ]:
if DEVICE == 'CPU':

    strategy = tf.distribute.get_strategy()
    print('\nUsing Default Distribution Strategy  for CPU')


if DEVICE == 'GPU':

    gpu_accelerarors = tf.config.list_physical_devices('GPU')
        
    if len(gpu_accelerarors) > 1:
        strategy = tf.distribute.MirroredStrategy()
        print(f'Number of GPUs available: {len(gpu_accelerarors)}')
        print('\n Using Mirrored Distribution Strategy')
        
    else:
        strategy = tf.distribute.get_strategy()
        if len(gpu_accelerarors) == 1:
            print(f'Number of GPUs available: 1')
            print('\nUsing Default Distribution Strategy for GPU')
        else:
            print('ERROR: GPU not available')
            print('\nUsing Default Distribution Strategy  for CPU')
        
if DEVICE == 'TPU':

    try:
        resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(resolver)
        tf.tpu.experimental.initialize_tpu_system(resolver)
        strategy = tf.distribute.experimental.TPUStrategy(resolver)
        tpu_accelerarors = tf.config.list_logical_devices('TPU')
        print(f'Number of TPU cores available: {len(tpu_accelerarors)}')
        print(f'\nUsing TPU Distribution Strategy')
        
    except:
        print('ERROR: TPU not available')
        print('\nUsing Default Distribution Strategy for CPU')
        strategy = tf.distribute.get_strategy()
        
        
if ENABLE_MIXED_PRECISION:
    
    print('\nMixed Precision enabled:')
    
    if DEVICE == 'GPU':
        policy = mixed_precision.Policy('mixed_float16')
        
    if DEVICE == 'TPU':
        policy = mixed_precision.Policy('mixed_bfloat16')
        
    mixed_precision.set_policy(policy)
    
    print('\t...Compute dtype: %s' % policy.compute_dtype)
    print('\t...Variable dtype: %s' % policy.variable_dtype)


REPLICAS = strategy.num_replicas_in_sync
print(f'\nREPLICAS: {REPLICAS}')

Number of TPU cores available: 8

Using TPU Distribution Strategy

Mixed Precision enabled:
	...Compute dtype: bfloat16
	...Variable dtype: float32

REPLICAS: 8


## Helper functions
- 클래스 Dataset : 
    - 클래스 변수 : tf.io.FixedLenFeature([], tf.string) : 고정길이 입력기능 (컬럼별로 지정)
    - parse_function, generator : [TFRecord 파일 사용](https://limjun92.github.io/assets/TensorFlow%202.0%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC/3.%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EB%A1%9C%EB%93%9C%20%EB%B0%8F%20%EC%82%AC%EC%A0%84%20%EC%B2%98%EB%A6%AC/%5B%ED%8A%9C%ED%86%A0%EB%A6%AC%EC%96%BC8%5D%20TFRecord%EC%99%80%20tf.Example/)
    - augment_function : 상하반전, 좌우반전, 채도조절, 명도조절
    
- create_model
    ```
    model = tf.keras.Sequential([
    effnets[EFF_NET](
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights='imagenet',
        include_top=False,
        drop_connect_rate=0.7),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(11, activation='sigmoid')
    ])
    
    ```
- compile_model
    ```
    model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
    metrics=[tf.keras.metrics.AUC(name='auc', multi_label=True)])
    
    ```


In [ ]:
class Dataset:
    
    feature_description = {
        "StudyInstanceUID"           : tf.io.FixedLenFeature([], tf.string),
        "image"                      : tf.io.FixedLenFeature([], tf.string),
        "ETT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "ETT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Incompletely Imaged"  : tf.io.FixedLenFeature([], tf.int64), 
        "NGT - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Abnormal"             : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Borderline"           : tf.io.FixedLenFeature([], tf.int64), 
        "CVC - Normal"               : tf.io.FixedLenFeature([], tf.int64), 
        "Swan Ganz Catheter Present" : tf.io.FixedLenFeature([], tf.int64),
    }
    
    def __init__(self, image_size):
        self.image_size = image_size
        
    def parse_function(self, example_proto):
        example = tf.io.parse_single_example(example_proto, self.feature_description)
        image = tf.io.decode_image(example['image'], channels=3)
        label = [example['ETT - Abnormal'],
                 example['ETT - Borderline'],
                 example['ETT - Normal'],
                 example['NGT - Abnormal'],
                 example['NGT - Borderline'],
                 example['NGT - Incompletely Imaged'],
                 example['NGT - Normal'],
                 example['CVC - Abnormal'],
                 example['CVC - Borderline'],
                 example['CVC - Normal'],
                 example['Swan Ganz Catheter Present']]
        return image, label 
    
    def augment_function(self, image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_contrast(image, 0.8, 1.2)
        image = tf.image.random_brightness(image, 0.1)   
        return image, label 
    
    def process_function(self, image, label):
        image.set_shape([None, self.image_size, self.image_size, 3])
        label.set_shape([None, 11])
        image = tf.image.resize(image, [self.image_size, self.image_size], 'bilinear')/255
        return image, label
            
    def generator(self, files, batch_size=1, repeat=False, augment=False, shuffle=True):
        AUTO = tf.data.experimental.AUTOTUNE
        ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
        if shuffle: 
            opt = tf.data.Options()
            opt.experimental_deterministic = False
            ds = ds.with_options(opt)
            ds = ds.shuffle(2000)
        ds = ds.map(self.parse_function, num_parallel_calls=AUTO)
        if repeat:
            ds = ds.repeat()
        if augment:
            ds = ds.map(self.augment_function, num_parallel_calls=AUTO)
        ds = ds.batch(batch_size)
        ds = ds.map(self.process_function, num_parallel_calls=AUTO)
        ds = ds.prefetch(AUTO)
        return ds

In [ ]:
def create_model(name, input_shape, classes, output_bias=None):
    
    # Dictionary mapping name to model function
    
    EFFICIENT_NETS = {'B0': efn.EfficientNetB0, 
                      'B1': efn.EfficientNetB1, 
                      'B2': efn.EfficientNetB2, 
                      'B3': efn.EfficientNetB3, 
                      'B4': efn.EfficientNetB4, 
                      'B5': efn.EfficientNetB5, 
                      'B6': efn.EfficientNetB6,
                      'B7': efn.EfficientNetB7}
    
    # Output layer bias initialization
    
    if output_bias is None:
        output_bias = 'zeros'
    else:
        output_bias = tf.keras.initializers.Constant(output_bias)
        
    
    # Base model
    
    base_model = EFFICIENT_NETS[name](include_top=False, 
                                      weights='imagenet', 
                                      input_shape=input_shape)
    
    # Model
    
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(classes, bias_initializer=output_bias)(x)
    outputs = tf.keras.layers.Activation('sigmoid', dtype='float32')(x) # Supports mixed-precision training
    
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
def compile_model(model, lr=0.0001):
    
    optimizer = tf.keras.optimizers.Adam(lr=lr)
    
    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05)
        
    metrics = [tf.keras.metrics.AUC(name='auc')]

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

### Callbacks
- 모델 저장 (Validation AUC max 일 때)
    - [tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)
- 모델의 개선이 없을 경우, Learning Rate를 조절해 모델의 개선을 유도
    - [tf.keras.callbacks.ReduceLROnPlateau](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ReduceLROnPlateau) [[detail](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)]
- 모니터링 되는 모델의 지표 개선이 멈췄을 때 학습 중지
    - [tf.keras.callbacks.EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)




In [ ]:
def create_callbacks(model_save_path, fold, verbose=1):
    
    verbose = int(verbose>0)
    
    if not os.path.exists(model_save_path):
        os.makedirs(model_save_path)
    
    cpk_path = f'{model_save_path}/model-f{fold}.h5'

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        filepath=cpk_path,
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=verbose
    )

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_auc',
        mode='max',
        factor=0.1,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=10, 
        verbose=verbose
    )
    
    
    
    callbacks = [checkpoint, reducelr, earlystop]
    
    return callbacks

#### count_items : 에포크의 스텝수 세는데 사용

In [ ]:
def count_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

## Main Training Pipeline

- (현재) 교차검증이 아닌 한번만 학습하는 코드

In [ ]:
folds_val_auc = [None] * FOLDS # Store the validation auc for each fold

skf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

print(f'Training...')

for fold, (train_idx, valid_idx) in enumerate(skf.split(np.arange(NUM_TF_RECS))):
    
    print(f'\n\n{"*"*100} \nFOLD: {fold+1}')
    
    # Input Pipeline ******************************************************
    
    train_files = tf.io.gfile.glob(f'{DATA_PATH}/train_tfrecords/{idx:02}*.tfrec' for idx in train_idx)
    valid_files = tf.io.gfile.glob(f'{DATA_PATH}/train_tfrecords/{idx:02}*.tfrec' for idx in valid_idx)
    
    ds = Dataset(IMG_SIZE)
    
    train_ds = ds.generator(train_files, 
                            BATCH_SIZE, #*REPLICAS, 
                            repeat=True, 
                            augment=True, 
                            shuffle=True)

    valid_ds = ds.generator(valid_files, 
                            BATCH_SIZE, #*REPLICAS,  
                            repeat=False, 
                            augment=False, 
                            shuffle=False)
    
    # Calculate the steps_per_epoch
    
    steps_per_epoch = count_items(train_files)//(BATCH_SIZE) *2 #*REPLICAS) * 2
    
    
    # Build Model ******************************************************
    
    tf.keras.backend.clear_session()
        
    with strategy.scope():
        model = create_model(name=EFF_NET, 
                             input_shape=(IMG_SIZE,IMG_SIZE,3), 
                             classes=11)

        model = compile_model(model, lr=0.0001)
        model.summary()
        
    print(f'\nModel initialized and compiled: EfficientNet-{EFF_NET}')
    
        
    # Train ******************************************************
    
    callbacks = create_callbacks(MODEL_PATH, fold+1, verbose=VERBOSE)
                                # MODEL_PATH = '/kaggle/working/models'

    print(f'\nModel training...\n')
    
    history = model.fit(train_ds, 
                        epochs=EPOCHS, 
                        steps_per_epoch=steps_per_epoch,
                        validation_data=valid_ds, 
                        callbacks=callbacks,
                        verbose=VERBOSE)
    
    # Save acc for each fold in a list
    folds_val_auc[fold] = max(history.history['val_auc'])
    
    print(f'\nModel trained \n\nFOLD-{fold+1} Validation AUC = {folds_val_auc[fold]}')
    
    break

Training...


**************************************************************************************************** 
FOLD: 1
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
efficientnet-b0 (Functional) (None, 7, 7, 1280)        4049564   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 11)                14091     
_________________________________________________________________
activation (Activation)      (None, 11)                0         
Total params: 4,063,655
Trainable params: 4,021,639
Non-trainable params: 42,016
______________________________________________________

## 학습훈련 그래프

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history['auc'], 'r', label='train auc')
    plt.plot(hist.history['val_auc'], 'g', label='val auc')
    plt.title("model auc")
    plt.ylabel("auc")
    plt.xlabel("epoch")
    plt.legend(loc='upper left')
    plt.show()


plot_hist(history)

In [ ]:
hist.history.key()

In [ ]:
hist_df = pd.DataFrame(hist.history) 
hist_df

In [ ]:
hist_df.describe()

In [ ]:
hist_df.to_csv("hist_{EFF_NET}.csv", mode='w')

In [ ]:
from IPython.display import FileLink
FileLink(r'df_name.csv')